# DataSet load and Training

In [ ]:
import numpy as np
import re
import pandas as pd
import time
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score
# Clean text data
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


Load the Train, Test and Validation Data Sets

In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/DataSet/data_processed_train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/DataSet/data_processed_test.csv")
val_data = pd.read_csv("/content/drive/MyDrive/DataSet/data_processed_validate.csv")

In [ ]:
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # # Remove urls
    # text = re.sub(r'http\S+', '', text)
    # # Remove mentions
    text = re.sub(r'@\S+', '', text)
    # # Remove numbers
    text = re.sub(r'\d+', '', text)
    # # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # # Remove stopwords and lemmatize
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

train_data['text'] = train_data['text'].apply(clean_text)
test_data['text'] = test_data['text'].apply(clean_text)
val_data['text'] = val_data['text'].apply(clean_text)

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(10000, 2)
(5000, 2)
(2000, 2)


Check the data set is balanced or not

In [ ]:
print(train_data['target'].value_counts())
print(test_data['target'].value_counts())
print(val_data['target'].value_counts())

-1    5014
 1    4986
Name: target, dtype: int64
-1    2521
 1    2479
Name: target, dtype: int64
-1    1012
 1     988
Name: target, dtype: int64


Check Null Values present or not?

In [ ]:
print(train_data.isnull().sum())
print(test_data.isnull().sum())
print(val_data.isnull().sum())

target    0
text      0
dtype: int64
target    0
text      0
dtype: int64
target    0
text      0
dtype: int64


In [ ]:
# train_data = train_data.dropna()
# test_data = test_data.dropna()
# val_data = val_data.dropna()

# Balancing the DataSets

In [ ]:
# Balancing train data
counts = train_data['target'].value_counts()
min_count = counts.min()

train_data = pd.concat([train_data[train_data['target'] == t].sample(min_count) for t in counts.index])

In [ ]:
# balancing validation dataset
counts = val_data['target'].value_counts()
min_count = counts.min()

val_data = pd.concat([val_data[val_data['target'] == t].sample(min_count) for t in counts.index])

In [ ]:
# balancing Test dataset
counts = test_data['target'].value_counts()
min_count = counts.min()

test_data = pd.concat([test_data[test_data['target'] == t].sample(min_count) for t in counts.index])

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(9972, 2)
(4958, 2)
(1976, 2)


To check datasets are balanced or not?

In [ ]:
print(train_data['target'].value_counts())
print(test_data['target'].value_counts())
print(val_data['target'].value_counts())

-1    4986
 1    4986
Name: target, dtype: int64
-1    2479
 1    2479
Name: target, dtype: int64
-1    988
 1    988
Name: target, dtype: int64


# **Predictions Using CountVectorizer** 

In [ ]:
start_time = time.time()
# Create document term matrix using CountVectorizer
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(train_data['text'])

X_test_count = count_vectorizer.transform(test_data['text'])

# Define the pipeline for CountVectorizer
pipeline_count = Pipeline([
    ('clf', LogisticRegression())
])

# Fit the model using CountVectorizer
pipeline_count.fit(X_train_count, train_data['target'])
end_time = time.time()
print("Time taken to train the model: ", end_time - start_time, " seconds")

start_time = time.time()
y_pred_count = pipeline_count.predict(X_test_count)
end_time = time.time()
print("Time taken to test the model: ", end_time - start_time, " seconds")
f1_score_count = f1_score(test_data['target'], y_pred_count)

# Print the confusion matrix with labels

cm = confusion_matrix(test_data['target'], y_pred_count)
print("Confusion Matrix:")
print(cm)

# Print the f1 score for CountVectorizer and TfidfVectorizer
print("f1 score using CountVectorizer: ", f1_score_count)


# Time taken to train the model:  1.951298475265503  seconds
# Time taken to test the model:  0.0012433528900146484  seconds
# Confusion Matrix:
# [[1782  697]
#  [ 668 1811]]
# f1 score using CountVectorizer:  0.726288349709244

# Predictions Using TfidfVectorizer

In [ ]:
start_time = time.time()
# Create document term matrix using TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['text'])


# Define the pipeline for TfidfVectorizer
pipeline_tfidf = Pipeline([
    ('clf', LogisticRegression())
])

# Fit the model using TfidfVectorizer
pipeline_tfidf.fit(X_train_tfidf, train_data['target'])
end_time = time.time()
print("Time taken to train the model: ", end_time - start_time, " seconds")

start_time = time.time()
X_test_tfidf = tfidf_vectorizer.transform(test_data['text'])
y_pred_tfidf = pipeline_tfidf.predict(X_test_tfidf)
end_time = time.time()
print("Time taken to test the model: ", end_time - start_time, " seconds")

f1_score_tfidf = f1_score(test_data['target'], y_pred_tfidf)

# precision, recall, f1_score, _ = precision_recall_fscore_support(test_data['target'], y_pred_tfidf)

# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 score:", f1_score)

cm = confusion_matrix(test_data['target'], y_pred_tfidf)
print("Confusion Matrix:")
print(cm)


print("f1 score using TfidfVectorizer: ", f1_score_tfidf)


# Time taken to train the model:  0.7965173721313477  seconds
# Time taken to test the model:  0.2213137149810791  seconds
# Confusion Matrix:
# [[1798  681]
#  [ 664 1815]]
# f1 score using TfidfVectorizer:  0.7296482412060302

# Hyper parameter tunning for Countvectorizer

In [ ]:
from sklearn.metrics import accuracy_score

# Create document term matrix using CountVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,1))

# Define the pipeline for CountVectorizer
# Define the pipeline for CountVectorizer with hyperparameters to tune
pipeline_count = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LogisticRegression())
])

# Define hyperparameters to tune for CountVectorizer
param_grid_count = {    
    'clf__C': np.logspace(-1, 1, 20),
    'clf__penalty': ['l1','l2'],
    'clf__max_iter': [10,100],
    'clf__solver': ['liblinear']
}
# param_grid_count = {
#     'clf__C': np.logspace(-3, 3, 7),
#     'clf__penalty': ['l1', 'l2'],
#     'clf__dual': [False, True],
#     'clf__fit_intercept': [True, False],
#     'clf__intercept_scaling': [1, 2, 3],
#     'clf__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
#     'clf__max_iter': [100],
#     'clf__class_weight': [None, 'balanced']
# }

# Perform hyperparameter tuning using GridSearchCV with the validation set
grid_search_count = GridSearchCV(pipeline_count, param_grid=param_grid_count, cv=5)
grid_search_count.fit(val_data['text'], val_data['target'])

for i in range(len(grid_search_count.cv_results_['params'])):
    print(grid_search_count.cv_results_['params'][i], " -> ", grid_search_count.cv_results_['mean_test_score'][i])

# Print the best hyperparameters for CountVectorizer
print("Best hyperparameters for CountVectorizer: ", grid_search_count.best_params_)

# Make predictions using the best CountVectorizer model on the validation set
best_count_model = grid_search_count.best_estimator_
print(best_count_model)

y_pred = best_count_model.predict(val_data['text'])

accuracy = accuracy_score(val_data['target'], y_pred)

# Print the accuracy of the best CountVectorizer model on the validation set
print("Accuracy of the best CountVectorizer model on the validation set: ", accuracy)


In [ ]:
# Best hyperparameters for CountVectorizer:  {'clf__C': 0.5455594781168519, 'clf__max_iter': 10, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}

# Accuracy of the best CountVectorizer model on the validation set:  0.9271255060728745

# **Hyper Parameter tunninng for TfidfVectorizer**

In [ ]:
# Create document term matrix using TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))

# Define the pipeline for TfidfVectorizer with hyperparameters to tune
pipeline_tfidf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

# Define hyperparameters to tune for TfidfVectorizer
param_grid_tfidf = {
    
     #'clf__C': np.logspace(-1, 1, 100),
#     # 'tfidf__max_df': [0.5, 0.75, 1.0],
    'clf__C': [0.001,1,10],
    'clf__max_iter': [10,100,200],
    'clf__penalty': ['l1', 'l2'],
    'clf__solver': [ 'liblinear']
}
# param_grid_tfidf = {
#     'clf__C': np.logspace(-3, 3, 7),
#     'clf__penalty': ['l1', 'l2'],
#     'clf__dual': [False, True],
#     'clf__fit_intercept': [True, False],
#     'clf__intercept_scaling': [1, 2, 3],
#     'clf__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
#     'clf__max_iter': [100],
#     'clf__class_weight': [None, 'balanced']
# }

# Perform hyperparameter tuning using GridSearchCV with the validation set
grid_search_tfidf = GridSearchCV(pipeline_tfidf, param_grid=param_grid_tfidf, cv=5)
grid_search_tfidf.fit(val_data['text'], val_data['target'])

# Print accuracy for each set of hyperparameters
print("Accuracy for each set of hyperparameters: ")
for i in range(len(grid_search_tfidf.cv_results_['params'])):
    print(grid_search_tfidf.cv_results_['params'][i], " -> ", grid_search_tfidf.cv_results_['mean_test_score'][i])

# Print the best hyperparameters for TfidfVectorizer
print("Best hyperparameters for TfidfVectorizer: ", grid_search_tfidf.best_params_)

# Make predictions using the best TfidfVectorizer model on the validation set
best_tfidf_model = grid_search_tfidf.best_estimator_
print(best_tfidf_model)


# Print the best hyperparameters for TfidfVectorizer
print("Best hyperparameters for TfidfVectorizer: ", grid_search_tfidf.best_params_)



t_pred = best_tfidf_model.predict(val_data['text'])

accuracy = accuracy_score(val_data['target'], t_pred)

# Print the accuracy of the best CountVectorizer model on the validation set
print("Accuracy of the best CountVectorizer model on the validation set: ", accuracy)


In [ ]:
# Best hyperparameters for TfidfVectorizer:  {'clf__C': 1, 'clf__max_iter': 10, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}
# Accuracy of the best CountVectorizer model on the validation set:  0.9129554655870445

# **CountVectorizer with best hyperparameters**

In [ ]:
start_time = time.time()
# Create document term matrix using CountVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,1))
X_train_count = count_vectorizer.fit_transform(train_data['text'])


C = 0.42813323987193935
max_iter = 10
penalty = 'l2'
solver = 'liblinear'

pipeline_count = Pipeline([
    ('clf', LogisticRegression(solver=solver, max_iter=max_iter, C=C, penalty=penalty))
])


# Define the pipeline for CountVectorizer
# pipeline_count = Pipeline([
#     ('clf', LogisticRegression(solver=grid_search_count.best_params_['clf__solver'],max_iter=grid_search_count.best_params_['clf__max_iter'], C=grid_search_count.best_params_['clf__C'], penalty=grid_search_count.best_params_['clf__penalty']))
    
# ])


# Fit the model using CountVectorizer
pipeline_count.fit(X_train_count, train_data['target'])
end_time = time.time()
print("Time taken to train the model: ", end_time - start_time, " seconds")

start_time = time.time()
X_test_count = count_vectorizer.transform(test_data['text'])
y_pred_count = pipeline_count.predict(X_test_count)
end_time = time.time()
print("Time taken to test the model: ", end_time - start_time, " seconds")
# Generate classification report for CountVectorizer
report_count = classification_report(test_data['target'], y_pred_count)
print("Classification report using CountVectorizer:\n", report_count)


f1_score_count = f1_score(test_data['target'], y_pred_count)

cm = confusion_matrix(test_data['target'], y_pred_count)
print("Confusion Matrix:")
print(cm)

precision_count = precision_score(test_data['target'], y_pred_count)
recall_count = recall_score(test_data['target'], y_pred_count)

print("Precision using CountVectorizer: ", precision_count)
print("Recall using CountVectorizer: ", recall_count)


# Print the f1 score for CountVectorizer 
print("f1 score using CountVectorizer: ", f1_score_count)


# Time taken to train the model:  0.1705327033996582  seconds
# Time taken to test the model:  0.07443976402282715  seconds
# Classification report using CountVectorizer:
#                precision    recall  f1-score   support

#           -1       0.73      0.72      0.73      2479
#            1       0.72      0.74      0.73      2479

#     accuracy                           0.73      4958
#    macro avg       0.73      0.73      0.73      4958
# weighted avg       0.73      0.73      0.73      4958

# Confusion Matrix:
# [[1780  699]
#  [ 647 1832]]
# Precision using CountVectorizer:  0.723824575266693
# Recall using CountVectorizer:  0.7390076643807987
# f1 score using CountVectorizer:  0.7313373253493014


# **TfiDf Vectorizer with best hyperparameters**

In [ ]:
start_time = time.time()
# Create document term matrix using TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['text'])


# Define the pipeline for TfidfVectorizer with best hyperparameters
pipeline_tfidf_lr_best = Pipeline([
    
    
('clf', LogisticRegression(solver=grid_search_tfidf.best_params_['clf__solver'], 
                            C=grid_search_tfidf.best_params_['clf__C'], 
                            penalty=grid_search_tfidf.best_params_['clf__penalty'],
                            max_iter=grid_search_tfidf.best_params_['clf__max_iter']))
])

# Fit the model using TfidfVectorizer with best hyperparameters
pipeline_tfidf_lr_best.fit(X_train_tfidf, train_data['target'])
end_time = time.time()
print("Time taken to train the model: ", end_time - start_time, " seconds")

start_time = time.time()
X_test_tfidf = tfidf_vectorizer.transform(test_data['text'])
# Predict using the test data
y_pred_tfidf = pipeline_tfidf_lr_best.predict(X_test_tfidf)

end_time = time.time()
print("Time taken to test the model: ", end_time - start_time, " seconds")

# Generate classification report for TfidfVectorizer
report_count = classification_report(test_data['target'], y_pred_tfidf)
print("Classification report using tfidfVectorizer:\n", report_count)


cm = confusion_matrix(test_data['target'], y_pred_tfidf)
print("Confusion Matrix:")
print(cm)

precision_count = precision_score(test_data['target'], y_pred_tfidf)
recall_count = recall_score(test_data['target'], y_pred_tfidf)

print("Precision using TfIDfVectorizer: ", precision_count)
print("Recall using TfiDfVectorizer: ", recall_count)


# Calculate f1 score
f1_score_tfidf = f1_score(test_data['target'], y_pred_tfidf)

# Print the f1 score
print("f1 score using TfidfVectorizer with best hyperparameters: ", f1_score_tfidf)

# Time taken to train the model:  0.22537684440612793  seconds
# Time taken to test the model:  0.13890576362609863  seconds
# Classification report using tfidfVectorizer:
#                precision    recall  f1-score   support

#           -1       0.73      0.73      0.73      2479
#            1       0.73      0.73      0.73      2479

#     accuracy                           0.73      4958
#    macro avg       0.73      0.73      0.73      4958
# weighted avg       0.73      0.73      0.73      4958

# Confusion Matrix:
# [[1799  680]
#  [ 664 1815]]
# Precision using TfIDfVectorizer:  0.7274549098196392
# Recall using TfiDfVectorizer:  0.7321500605082695
# f1 score using TfidfVectorizer with best hyperparameters:  0.7297949336550059


# **Measuring** Bias


Loading EEC corpus data


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/DataSet/Equity-Evaluation-Corpus.csv")

In [ ]:
data['Sentence'].nunique() # Number of unique sentences

In [ ]:
data.isnull().sum()

In [ ]:
data.columns

In [ ]:
data.head(10)

In [ ]:
data.shape

In [ ]:
def clean_text(text):
#     # Convert text to lowercase
    text = text.lower()
#     # Remove urls
#     text = re.sub(r'http\S+', '', text)
#     # Remove mentions
#     text = re.sub(r'@\S+', '', text)
#     # Remove numbers
#     text = re.sub(r'\d+', '', text)
#     # Remove punctuation
#     text = re.sub(r'[^\w\s]', '', text)
#     # Remove stopwords and lemmatize
#     # text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

data['Sentence'] = data['Sentence'].apply(clean_text)

In [ ]:
# Group the data by gender
gender_groups = data.groupby('Gender')

# Create separate datasets for male and female
male_data = gender_groups.get_group('male')
female_data = gender_groups.get_group('female')


Both male and female data sets are same at every index, just gender is different in the sentence

In [ ]:
print(male_data['Sentence'].iloc[100:105])
print(female_data['Sentence'].iloc[100:105])

Collect data set for each emotion from the male dataset

In [ ]:

male_emotion_groups = male_data.groupby('Emotion')

# Create separate datasets for male and female
anger_male_data = male_emotion_groups.get_group('anger')
sadness_male_data=male_emotion_groups.get_group('sadness')
fear_male_data=male_emotion_groups.get_group('fear')
joy_male_data=male_emotion_groups.get_group('joy')

Collect data set for each emotion from the female dataset

In [ ]:
female_emotion_groups = female_data.groupby('Emotion')

# Create separate datasets for male and female
anger_female_data=female_emotion_groups.get_group('anger')
sadness_female_data=female_emotion_groups.get_group('sadness')
fear_female_data=female_emotion_groups.get_group('fear')
joy_female_data=female_emotion_groups.get_group('joy')

In [ ]:
print(anger_male_data.shape)
print(sadness_male_data.shape)
print(fear_male_data.shape)
print(joy_male_data.shape)


In [ ]:
print(anger_female_data.shape)
print(sadness_female_data.shape)
print(fear_female_data.shape)
print(joy_female_data.shape)

The text in every index or row for each emotion was same for male and female and gender was different

In [ ]:
print(anger_male_data['Sentence'].iloc[900:902])
print(anger_female_data['Sentence'].iloc[900:902])
print("\n")
print(fear_male_data['Sentence'].iloc[510:512])
print(fear_female_data['Sentence'].iloc[510:512])
print("\n")
print(sadness_male_data['Sentence'].iloc[510:512])
print(sadness_female_data['Sentence'].iloc[510:512])
print("\n")
print(joy_male_data['Sentence'].iloc[700:702])
print(joy_female_data['Sentence'].iloc[700:702])

No Emotion word datasets for male and female

In [ ]:
# data_null_emotion = data[data['Emotion'].isnull()]

# data_null_emotion.shape

# # Group the data by gender
# No_emotion_gender_groups = data_null_emotion.groupby('Gender')

# # Create separate datasets for male and female
# No_emotion_male_data = No_emotion_gender_groups.get_group('male')
# No_emotion_female_data = No_emotion_gender_groups.get_group('female')
# print(No_emotion_female_data.shape)
# print(No_emotion_male_data.shape)



Null Race


In [ ]:
# data_null_Race = data[data['Race'].isnull()]
# data_null_Race.shape
# No_Race_Gender_group=data_null_Race.groupby('Gender')
# No_race_male_data=No_Race_Gender_group.get_group('male')
# No_race_female_data=No_Race_Gender_group.get_group('female')

# print(No_race_female_data.shape)
# print(No_race_male_data.shape)

# print(No_race_female_data['Sentence'].iloc[100:101])
# print(No_race_male_data['Sentence'].iloc[100:101])




# **Using TfIdf Vectorizer**

pipeline_tfidf_lr_best-- model name after hyperparameter tuning

# Bias checking for anger emotion for male and female datasets

Anger Male


In [ ]:
class_names = pipeline_tfidf_lr_best.classes_

In [ ]:
new_anger_male=anger_male_data[['Sentence']]
new_anger_male.shape
anger_male_test = tfidf_vectorizer.transform(new_anger_male['Sentence'])
anger_male_pred = pipeline_tfidf_lr_best.predict_proba(anger_male_test)
anger_male_pred
for i in range(len(anger_male_pred)):
    print(f"Observation {i}: {class_names[0]}={anger_male_pred[i][0]}, {class_names[1]}={anger_male_pred[i][1]}")

Anger Female 

In [ ]:
new_anger_female=anger_female_data[['Sentence']]
anger_female_test= tfidf_vectorizer.transform(new_anger_female['Sentence'])
anger_female_pred= pipeline_tfidf_lr_best.predict_proba(anger_female_test)
anger_female_pred
for i in range(len(anger_female_pred)):
     print(f"Observation {i}: {class_names[0]}={anger_female_pred[i][0]}, {class_names[1]}={anger_female_pred[i][1]}")

Checking the intensity average for anger emotion and checking bias

In [ ]:
diff_anger = []
diff_anger_count = 0  # number of pairs with differences
same_anger_count = 0  # number of pairs without differences
anger_male_prob_sum=0
anger_female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(anger_female_pred)):
          female_prob = anger_female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = anger_male_pred[i][1]  # probability of positive class for male sentence i
          anger_male_prob_sum+=male_prob # male probabilities sum
          anger_female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          diff_anger.append(diff)
          if diff > 0:
              diff_anger_count += 1
          else:
              same_anger_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_anger_count > 0:
    avg_diff = sum(diff_anger) / diff_anger_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_anger_count)
print("Number of pairs without differences:", same_anger_count)
print("Average difference:", avg_diff)

avg_anger_male_scores=(anger_male_prob_sum)/len(anger_male_pred)
avg_anger_female_scores=(anger_female_prob_sum)/len(anger_female_pred)
print("Male anger Average",avg_anger_male_scores)
print("Female anger Average",avg_anger_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_anger_female_scores > avg_anger_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# Bias checking for sadness emotion for male and female datasets

Sadness Male

In [ ]:
new_sadness_male=sadness_male_data[['Sentence']]
new_sadness_male.shape
sadness_male_test = tfidf_vectorizer.transform(new_sadness_male['Sentence'])
sadness_male_pred = pipeline_tfidf_lr_best.predict_proba(sadness_male_test)
sadness_male_pred
for i in range(len(sadness_male_pred)):
    print(f"Observation {i}: {class_names[0]}={sadness_male_pred[i][0]}, {class_names[1]}={sadness_male_pred[i][1]}")

Sadness Female

In [ ]:
new_sadness_female=sadness_female_data[['Sentence']]
sadness_female_test= tfidf_vectorizer.transform(new_sadness_female['Sentence'])
sadness_female_pred= pipeline_tfidf_lr_best.predict_proba(sadness_female_test)
sadness_female_pred
for i in range(len(sadness_female_pred)):
    print(f"Observation {i}: {class_names[0]}={sadness_female_pred[i][0]}, {class_names[1]}={sadness_female_pred[i][1]}")

Checking the intensity average for sadness emotion and checking bias

In [ ]:
diff_sadness = []
diff_sadness_count = 0  # number of pairs with differences
same_sadness_count = 0  # number of pairs without differences
sadness_male_prob_sum=0
sadness_female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(sadness_female_pred)):
          female_prob = sadness_female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = sadness_male_pred[i][1]
            # probability of positive class for male sentence i
          sadness_male_prob_sum+=male_prob # male probabilities sum
          sadness_female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diff_sadness.append(diff)
          if diff > 0:
              diff_sadness_count += 1
          else:
              same_sadness_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_sadness_count > 0:
    avg_diff = sum(diff_sadness) / diff_sadness_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_sadness_count)
print("Number of pairs without differences:", same_sadness_count)
print("Average difference:", avg_diff)

avg_sadness_male_scores=(sadness_male_prob_sum)/len(sadness_male_pred)
avg_sadness_female_scores=(sadness_female_prob_sum)/len(sadness_female_pred)
print("Male sadness Average",avg_sadness_male_scores)
print("Female sadness Average",avg_sadness_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_sadness_female_scores > avg_sadness_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# Bias checking for joy emotion for male and female datasets

Joy male

In [ ]:
new_joy_male=joy_male_data[['Sentence']]
new_joy_male.shape
joy_male_test = tfidf_vectorizer.transform(new_joy_male['Sentence'])
joy_male_pred = pipeline_tfidf_lr_best.predict_proba(joy_male_test)
joy_male_pred
for i in range(len(joy_male_pred)):
    print(f"Observation {i}: {class_names[0]}={joy_male_pred[i][0]}, {class_names[1]}={joy_male_pred[i][1]}")

Joy female

In [ ]:
new_joy_female=joy_female_data[['Sentence']]
joy_female_test= tfidf_vectorizer.transform(new_joy_female['Sentence'])
joy_female_pred= pipeline_tfidf_lr_best.predict_proba(joy_female_test)
joy_female_pred
for i in range(len(joy_female_pred)):
    print(f"Observation {i}: {class_names[0]}={joy_female_pred[i][0]}, {class_names[1]}={joy_female_pred[i][1]}")

Checking the intensity average for joy emotion and checking bias

In [ ]:
diff_joy = []
diff_joy_count = 0  # number of pairs with differences
same_joy_count = 0  # number of pairs without differences
joy_male_prob_sum=0
joy_female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(joy_female_pred)):
          female_prob = joy_female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = joy_male_pred[i][1]
            # probability of positive class for male sentence i
          joy_male_prob_sum+=male_prob # male probabilities sum
          joy_female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diff_joy.append(diff)
          if diff > 0:
              diff_joy_count += 1
          else:
              same_joy_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_joy_count > 0:
    avg_diff = sum(diff_joy) / diff_joy_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_joy_count)
print("Number of pairs without differences:", same_joy_count)
print("Average difference:", avg_diff)

avg_joy_male_scores=(joy_male_prob_sum)/len(joy_male_pred)
avg_joy_female_scores=(joy_female_prob_sum)/len(joy_female_pred)
print("Male joy Average",avg_joy_male_scores)
print("Female joy Average",avg_joy_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_joy_female_scores > avg_joy_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# Bias checking for fear emotion for male and female datasets

fear male

In [ ]:
new_fear_male=fear_male_data[['Sentence']]
new_fear_male.shape
fear_male_test = tfidf_vectorizer.transform(new_fear_male['Sentence'])
fear_male_pred = pipeline_tfidf_lr_best.predict_proba(fear_male_test)
fear_male_pred
for i in range(len(fear_male_pred)):
    print(f"Observation {i}: {class_names[0]}={fear_male_pred[i][0]}, {class_names[1]}={fear_male_pred[i][1]}")

Fear Feamle

In [ ]:
new_fear_female=fear_female_data[['Sentence']]
fear_female_test= tfidf_vectorizer.transform(new_fear_female['Sentence'])
fear_female_pred= pipeline_tfidf_lr_best.predict_proba(fear_female_test)
fear_female_pred
for i in range(len(fear_female_pred)):
    print(f"Observation {i}: {class_names[0]}={fear_female_pred[i][0]}, {class_names[1]}={fear_female_pred[i][1]}")

Checking the intensity average for Fear emotion and checking bias

In [ ]:
diff_fear = []
diff_fear_count = 0  # number of pairs with differences
same_fear_count = 0  # number of pairs without differences
fear_male_prob_sum=0
fear_female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(fear_female_pred)):
          female_prob = fear_female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = fear_male_pred[i][1]
            # probability of positive class for male sentence i
          fear_male_prob_sum+=male_prob # male probabilities sum
          fear_female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diff_fear.append(diff)
          if diff > 0:
              diff_fear_count += 1
          else:
              same_fear_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_joy_count > 0:
    avg_diff = sum(diff_fear) / diff_fear_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_fear_count)
print("Number of pairs without differences:", same_fear_count)
print("Average difference:", avg_diff)

avg_fear_male_scores=(fear_male_prob_sum)/len(fear_male_pred)
avg_fear_female_scores=(fear_female_prob_sum)/len(fear_female_pred)
print("Male fear Average",avg_fear_male_scores)
print("Female fear Average",avg_fear_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_fear_female_scores > avg_fear_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# Checking the bias for entire male and female datsets

pipeline_tfidf_lr_best-- model name after hyperparameter tuning

Male EEC corpus data

In [ ]:
new_male_df = male_data[['Sentence']]
male_test = tfidf_vectorizer.transform(new_male_df['Sentence'])
#male_pred = pipeline_tfidf_lr_best.predict(male_test)
male_pred = pipeline_tfidf_lr_best.predict_proba(male_test)
male_pred
class_names = pipeline_tfidf_lr_best.classes_
print("Class names:", class_names)
for i in range(len(male_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

Female EEc Corpus data


In [ ]:
new_female_df = female_data[['Sentence']]
female_test = tfidf_vectorizer.transform(new_female_df['Sentence'])
female_pred = pipeline_tfidf_lr_best.predict_proba(female_test)
female_pred
class_names = pipeline_tfidf_lr_best.classes_
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

Checking the intensity Average and bias

In [ ]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print("The Average of male",avg_male_scores)
print("The Average of female",avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# Bias checking for Non-emotion datasets

In [ ]:
data_null_emotion = data[data['Emotion'].isnull()]

data_null_emotion.shape

# Group the data by gender
No_emotion_gender_groups = data_null_emotion.groupby('Gender')

# Create separate datasets for male and female
No_emotion_male_data = No_emotion_gender_groups.get_group('male')
No_emotion_female_data = No_emotion_gender_groups.get_group('female')
print(No_emotion_female_data.shape)
print(No_emotion_male_data.shape)

In [ ]:
print(No_emotion_male_data['Sentence'].iloc[10:15])
print(No_emotion_female_data['Sentence'].iloc[10:15])

In [ ]:
male_test = tfidf_vectorizer.transform(No_emotion_male_data['Sentence'])
male_pred = pipeline_tfidf_lr_best.predict_proba(male_test)
male_pred
class_names = pipeline_tfidf_lr_best.classes_
print("Class names:", class_names)
for i in range(len(male_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

In [ ]:
female_test = tfidf_vectorizer.transform(No_emotion_female_data['Sentence'])
female_pred = pipeline_tfidf_lr_best.predict_proba(female_test)
female_pred
class_names = pipeline_tfidf_lr_best.classes_
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

In [ ]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print("The Average of male",avg_male_scores)
print("The Average of female",avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# **Race** Dataset


In [ ]:
print(data['Race'].value_counts())
# Group the data by Race
Race_groups = data.groupby('Race')

# Create separate datasets for African-American and European
AfA_data = Race_groups.get_group('African-American')
Euro_data = Race_groups.get_group('European')


In [ ]:
print(AfA_data.shape)
print(Euro_data.shape)

print(AfA_data['Emotion'].value_counts())
print(Euro_data['Emotion'].value_counts())

In [ ]:
print(AfA_data['Sentence'].iloc[100:109])
print(Euro_data['Sentence'].iloc[100:109])

# Bias for Entire Race DataSets(AA and Eu)

African-American DataSet

In [ ]:
AfA_test = tfidf_vectorizer.transform(AfA_data['Sentence'])
AfA_pred = pipeline_tfidf_lr_best.predict_proba(AfA_test)
AfA_pred
class_names = pipeline_tfidf_lr_best.classes_
print("Class names:", class_names)
for i in range(len(AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={AfA_pred[i][0]}, {class_names[1]}={AfA_pred[i][1]}")

Euro Data Set


In [ ]:
Euro_test=tfidf_vectorizer.transform(Euro_data['Sentence'])
Euro_pred=pipeline_tfidf_lr_best.predict_proba(Euro_test)
Euro_pred
for i in range(len(Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={Euro_pred[i][0]}, {class_names[1]}={Euro_pred[i][1]}")

In [ ]:
diff_list = []
diff_count_race = 0  # number of pairs with differences
same_count_race = 0  # number of pairs without differences
Euro_prob_sum=0
AfA_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(Euro_pred)):
          Euro_prob = Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = AfA_pred[i][1]  # probability of positive class for AfA sentence i
          Euro_prob_sum+=Euro_prob # Euro probabilities sum
          AfA_prob_sum+=AfA_prob # AfA probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_list.append(diff)
          if diff > 0:
              diff_count_race += 1
          else:
              same_count_race += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count_race > 0:
    avg_diff_race = sum(diff_list) / diff_count_race
else:
    avg_diff_race = 0

print("Number of pairs with differences:", diff_count_race)
print("Number of pairs without differences:", same_count_race)
print("Average difference:", avg_diff_race)

In [ ]:
avg_AfA_scores=(AfA_prob_sum)/len(AfA_pred)
avg_Euro_scores=(Euro_prob_sum)/len(Euro_pred)

print(avg_AfA_scores)
print(avg_Euro_scores)

# Check if E=A not significant
if abs(avg_diff_race) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_Euro_scores > avg_AfA_scores:
    print("E↑–A↓ significant")
else:
    print("E↓–A↑ significant")

Bias Measure for Emtions in Race DataSet

In [ ]:
AfA_emotion_groups = AfA_data.groupby('Emotion')

# Create separate datasets for AfA and Euro
anger_AfA_data = AfA_emotion_groups.get_group('anger')
sadness_AfA_data=AfA_emotion_groups.get_group('sadness')
fear_AfA_data=AfA_emotion_groups.get_group('fear')
joy_AfA_data=AfA_emotion_groups.get_group('joy')

In [ ]:
Euro_emotion_groups = Euro_data.groupby('Emotion')

# Create separate datasets for AfA and Euro
anger_Euro_data = Euro_emotion_groups.get_group('anger')
sadness_Euro_data=Euro_emotion_groups.get_group('sadness')
fear_Euro_data=Euro_emotion_groups.get_group('fear')
joy_Euro_data=Euro_emotion_groups.get_group('joy')

In [ ]:
print(anger_AfA_data.shape)
print(anger_Euro_data.shape)

In [ ]:
print(sadness_AfA_data.shape)
print(sadness_Euro_data.shape)

In [ ]:
print(fear_AfA_data.shape)
print(fear_Euro_data.shape)

In [ ]:
print(joy_AfA_data.shape)
print(joy_Euro_data.shape)

The text in AfA and Euro data sets are same for every index or row, but Race was different

In [ ]:
print(anger_AfA_data['Sentence'].iloc[305:310])
print(anger_Euro_data['Sentence'].iloc[305:310])

# **Bias for Anger emotion for Race**

Anger AfA dataset

In [ ]:
new_anger_AfA=anger_AfA_data[['Sentence']]
new_anger_AfA.shape
anger_AfA_test = tfidf_vectorizer.transform(new_anger_AfA['Sentence'])
anger_AfA_pred = pipeline_tfidf_lr_best.predict_proba(anger_AfA_test)
anger_AfA_pred
for i in range(len(anger_AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={anger_AfA_pred[i][0]}, {class_names[1]}={anger_AfA_pred[i][1]}")

Anger Euro DataSet

In [ ]:
new_anger_Euro=anger_Euro_data[['Sentence']]
anger_Euro_test= tfidf_vectorizer.transform(new_anger_Euro['Sentence'])
anger_Euro_pred= pipeline_tfidf_lr_best.predict_proba(anger_Euro_test)
anger_Euro_pred
for i in range(len(anger_Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={anger_Euro_pred[i][0]}, {class_names[1]}={anger_Euro_pred[i][1]}")


Checking intensity score average and Bias

In [ ]:
diff_anger = []
diff_anger_count = 0  # number of pairs with differences
same_anger_count = 0  # number of pairs without differences
anger_AfA_prob_sum=0
anger_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(anger_Euro_pred)):
          Euro_prob = anger_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = anger_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          anger_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          anger_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_anger.append(diff)
          if diff > 0:
              diff_anger_count += 1
          else:
              same_anger_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_anger_count > 0:
    avg_diff = sum(diff_anger) / diff_anger_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_anger_count)
print("Number of pairs without differences:", same_anger_count)
print("Average difference:", avg_diff)

avg_anger_AfA_scores=(anger_AfA_prob_sum)/len(anger_AfA_pred)
avg_anger_Euro_scores=(anger_Euro_prob_sum)/len(anger_Euro_pred)
print("AfA anger Average",avg_anger_AfA_scores)
print("Euro anger Average",avg_anger_Euro_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_anger_Euro_scores > avg_anger_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

# **Sadness Bias for Race**

sadness AfA dataset

In [ ]:
new_sadness_AfA=sadness_AfA_data[['Sentence']]
new_sadness_AfA.shape
sadness_AfA_test = tfidf_vectorizer.transform(new_sadness_AfA['Sentence'])
sadness_AfA_pred = pipeline_tfidf_lr_best.predict_proba(sadness_AfA_test)
sadness_AfA_pred
for i in range(len(sadness_AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={sadness_AfA_pred[i][0]}, {class_names[1]}={sadness_AfA_pred[i][1]}")

Sadness Euro dataset

In [ ]:
new_sadness_Euro=sadness_Euro_data[['Sentence']]
sadness_Euro_test= tfidf_vectorizer.transform(new_sadness_Euro['Sentence'])
sadness_Euro_pred= pipeline_tfidf_lr_best.predict_proba(sadness_Euro_test)
sadness_Euro_pred
for i in range(len(sadness_Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={sadness_Euro_pred[i][0]}, {class_names[1]}={sadness_Euro_pred[i][1]}")


Checking intensity score average and bias

In [ ]:
diff_sadness = []
diff_sadness_count = 0  # number of pairs with differences
same_sadness_count = 0  # number of pairs without differences
sadness_AfA_prob_sum=0
sadness_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(sadness_Euro_pred)):
          Euro_prob = sadness_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = sadness_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          sadness_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          sadness_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_sadness.append(diff)
          if diff > 0:
              diff_sadness_count += 1
          else:
              same_sadness_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_sadness_count > 0:
    avg_diff = sum(diff_sadness) / diff_sadness_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_sadness_count)
print("Number of pairs without differences:", same_sadness_count)
print("Average difference:", avg_diff)

avg_sadness_AfA_scores=(sadness_AfA_prob_sum)/len(sadness_AfA_pred)
avg_sadness_Euro_scores=(sadness_Euro_prob_sum)/len(sadness_Euro_pred)
print("AfA sadness Average",avg_sadness_AfA_scores)
print("Euro sadness Average",avg_sadness_Euro_scores)

# Check if E=A not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_sadness_Euro_scores > avg_sadness_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

# **Joy Bias for Race**

joy AfA dataset

In [ ]:
new_joy_AfA=joy_AfA_data[['Sentence']]
new_joy_AfA.shape
joy_AfA_test = tfidf_vectorizer.transform(new_joy_AfA['Sentence'])
joy_AfA_pred = pipeline_tfidf_lr_best.predict_proba(joy_AfA_test)
joy_AfA_pred
for i in range(len(joy_AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={joy_AfA_pred[i][0]}, {class_names[1]}={joy_AfA_pred[i][1]}")

joy Euro dataset

In [ ]:
new_joy_Euro=joy_Euro_data[['Sentence']]
joy_Euro_test= tfidf_vectorizer.transform(new_joy_Euro['Sentence'])
joy_Euro_pred= pipeline_tfidf_lr_best.predict_proba(joy_Euro_test)
joy_Euro_pred
for i in range(len(joy_Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={joy_Euro_pred[i][0]}, {class_names[1]}={joy_Euro_pred[i][1]}")


Checking intensity score average and Bias

In [ ]:
diff_joy = []
diff_joy_count = 0  # number of pairs with differences
same_joy_count = 0  # number of pairs without differences
joy_AfA_prob_sum=0
joy_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(joy_Euro_pred)):
          Euro_prob = joy_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = joy_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          joy_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          joy_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_joy.append(diff)
          if diff > 0:
              diff_joy_count += 1
          else:
              same_joy_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_joy_count > 0:
    avg_diff = sum(diff_joy) / diff_joy_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_joy_count)
print("Number of pairs without differences:", same_joy_count)
print("Average difference:", avg_diff)

avg_joy_AfA_scores=(joy_AfA_prob_sum)/len(joy_AfA_pred)
avg_joy_Euro_scores=(joy_Euro_prob_sum)/len(joy_Euro_pred)
print("AfA joy Average",avg_joy_AfA_scores)
print("Euro joy Average",avg_joy_Euro_scores)

# Check if E=A not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_joy_Euro_scores > avg_joy_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

# Bias for fear emotion 

Fear AfA dataset

In [ ]:
new_fear_AfA=fear_AfA_data[['Sentence']]
new_fear_AfA.shape
fear_AfA_test = tfidf_vectorizer.transform(new_fear_AfA['Sentence'])
fear_AfA_pred = pipeline_tfidf_lr_best.predict_proba(fear_AfA_test)
fear_AfA_pred
for i in range(len(fear_AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={fear_AfA_pred[i][0]}, {class_names[1]}={fear_AfA_pred[i][1]}")

Fear Euro dataset

In [ ]:
new_fear_Euro=fear_Euro_data[['Sentence']]
fear_Euro_test= tfidf_vectorizer.transform(new_fear_Euro['Sentence'])
fear_Euro_pred= pipeline_tfidf_lr_best.predict_proba(fear_Euro_test)
fear_Euro_pred
for i in range(len(fear_Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={fear_Euro_pred[i][0]}, {class_names[1]}={fear_Euro_pred[i][1]}")



Checking intensity score average and bias

In [ ]:
diff_fear = []
diff_fear_count = 0  # number of pairs with differences
same_fear_count = 0  # number of pairs without differences
fear_AfA_prob_sum=0
fear_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(fear_Euro_pred)):
          Euro_prob = fear_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = fear_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          fear_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          fear_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_fear.append(diff)
          if diff > 0:
              diff_fear_count += 1
          else:
              same_fear_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_fear_count > 0:
    avg_diff = sum(diff_fear) / diff_fear_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_fear_count)
print("Number of pairs without differences:", same_fear_count)
print("Average difference:", avg_diff)

avg_fear_AfA_scores=(fear_AfA_prob_sum)/len(fear_AfA_pred)
avg_fear_Euro_scores=(fear_Euro_prob_sum)/len(fear_Euro_pred)
print("AfA fear Average",avg_fear_AfA_scores)
print("Euro fear Average",avg_fear_Euro_scores)

# Check if E=A not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_fear_Euro_scores > avg_fear_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

# Non- Race and Group by Gender 

In [ ]:
data_null_Race = data[data['Race'].isnull()]
data_null_Race.shape
No_Race_Gender_group=data_null_Race.groupby('Gender')
No_race_male_data=No_Race_Gender_group.get_group('male')
No_race_female_data=No_Race_Gender_group.get_group('female')

print(No_race_female_data.shape)
print(No_race_male_data.shape)

print(No_race_female_data['Sentence'].iloc[100:101])
print(No_race_male_data['Sentence'].iloc[100:101])

In [ ]:
male_test = tfidf_vectorizer.transform(No_race_male_data['Sentence'])
male_pred = pipeline_tfidf_lr_best.predict_proba(male_test)
male_pred
class_names = pipeline_tfidf_lr_best.classes_
print("Class names:", class_names)
for i in range(len(male_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

In [ ]:
female_test = tfidf_vectorizer.transform(No_race_female_data['Sentence'])
female_pred = pipeline_tfidf_lr_best.predict_proba(female_test)
female_pred
class_names = pipeline_tfidf_lr_best.classes_
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

In [ ]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print("The Average of male",avg_male_scores)
print("The Average of female",avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# **CountVectorizer**

pipeline_count --- the model after hyperparameter tuning

**Gender Bias**

male dataset

In [ ]:
new_male_df = male_data[['Sentence']]
male_test = count_vectorizer.transform(new_male_df['Sentence'])
#male_pred = pipeline_count.predict(male_test)
male_pred = pipeline_count.predict_proba(male_test)
male_pred
class_names =pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(male_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

Female Dataset

In [ ]:
new_female_df = female_data[['Sentence']]
female_test = count_vectorizer.transform(new_female_df['Sentence'])
female_pred = pipeline_count.predict_proba(female_test)
female_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

Checking intensity score average and bias

In [ ]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print("The Average of male",avg_male_scores)
print("The Average of female",avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# Anger dataset Bias for male and female

Anger male dataset

In [ ]:
new_anger_male=anger_male_data[['Sentence']]
new_anger_male.shape
anger_male_test = count_vectorizer.transform(new_anger_male['Sentence'])
anger_male_pred = pipeline_count.predict_proba(anger_male_test)
anger_male_pred
for i in range(len(anger_male_pred)):
    print(f"Observation {i}: {class_names[0]}={anger_male_pred[i][0]}, {class_names[1]}={anger_male_pred[i][1]}")

Anger female dataset

In [ ]:
new_anger_female=anger_female_data[['Sentence']]
anger_female_test= count_vectorizer.transform(new_anger_female['Sentence'])
anger_female_pred= pipeline_count.predict_proba(anger_female_test)
anger_female_pred
for i in range(len(anger_female_pred)):
     print(f"Observation {i}: {class_names[0]}={anger_female_pred[i][0]}, {class_names[1]}={anger_female_pred[i][1]}")

Checking intensity score average and bias

In [ ]:
diff_anger = []
diff_anger_count = 0  # number of pairs with differences
same_anger_count = 0  # number of pairs without differences
anger_male_prob_sum=0
anger_female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(anger_female_pred)):
          female_prob = anger_female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = anger_male_pred[i][1]
            # probability of positive class for male sentence i
          anger_male_prob_sum+=male_prob # male probabilities sum
          anger_female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diff_anger.append(diff)
          if diff > 0:
              diff_anger_count += 1
          else:
              same_anger_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_anger_count > 0:
    avg_diff = sum(diff_anger) / diff_anger_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_anger_count)
print("Number of pairs without differences:", same_anger_count)
print("Average difference:", avg_diff)

avg_anger_male_scores=(anger_male_prob_sum)/len(anger_male_pred)
avg_anger_female_scores=(anger_female_prob_sum)/len(anger_female_pred)
print("Male anger Average",avg_anger_male_scores)
print("Female anger Average",avg_anger_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_anger_female_scores > avg_anger_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# Sadness Bias for male and feamle 

male sadness dataset

In [ ]:
new_sadness_male=sadness_male_data[['Sentence']]
new_sadness_male.shape
sadness_male_test = count_vectorizer.transform(new_sadness_male['Sentence'])
sadness_male_pred = pipeline_count.predict_proba(sadness_male_test)
sadness_male_pred
for i in range(len(sadness_male_pred)):
    print(f"Observation {i}: {class_names[0]}={sadness_male_pred[i][0]}, {class_names[1]}={sadness_male_pred[i][1]}")

female sandness dataset

In [ ]:
new_sadness_female=sadness_female_data[['Sentence']]
sadness_female_test= count_vectorizer.transform(new_sadness_female['Sentence'])
sadness_female_pred= pipeline_count.predict_proba(sadness_female_test)
sadness_female_pred
for i in range(len(sadness_female_pred)):
    print(f"Observation {i}: {class_names[0]}={sadness_female_pred[i][0]}, {class_names[1]}={sadness_female_pred[i][1]}")

Checking the intensity score average and bias

In [ ]:
diff_sadness = []
diff_sadness_count = 0  # number of pairs with differences
same_sadness_count = 0  # number of pairs without differences
sadness_male_prob_sum=0
sadness_female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(sadness_female_pred)):
          female_prob = sadness_female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = sadness_male_pred[i][1]
            # probability of positive class for male sentence i
          sadness_male_prob_sum+=male_prob # male probabilities sum
          sadness_female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diff_sadness.append(diff)
          if diff > 0:
              diff_sadness_count += 1
          else:
              same_sadness_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_sadness_count > 0:
    avg_diff = sum(diff_sadness) / diff_sadness_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_sadness_count)
print("Number of pairs without differences:", same_sadness_count)
print("Average difference:", avg_diff)

avg_sadness_male_scores=(sadness_male_prob_sum)/len(sadness_male_pred)
avg_sadness_female_scores=(sadness_female_prob_sum)/len(sadness_female_pred)
print("Male sadness Average",avg_sadness_male_scores)
print("Female sadness Average",avg_sadness_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_sadness_female_scores > avg_sadness_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# Joy Bias for male and female

joy male dataset

In [ ]:
new_joy_male=joy_male_data[['Sentence']]
new_joy_male.shape
joy_male_test = count_vectorizer.transform(new_joy_male['Sentence'])
joy_male_pred = pipeline_count.predict_proba(joy_male_test)
joy_male_pred
for i in range(len(joy_male_pred)):
    print(f"Observation {i}: {class_names[0]}={joy_male_pred[i][0]}, {class_names[1]}={joy_male_pred[i][1]}")

Joy female dataset

In [ ]:
new_joy_female=joy_female_data[['Sentence']]
joy_female_test= count_vectorizer.transform(new_joy_female['Sentence'])
joy_female_pred= pipeline_count.predict_proba(joy_female_test)
joy_female_pred
for i in range(len(joy_female_pred)):
    print(f"Observation {i}: {class_names[0]}={joy_female_pred[i][0]}, {class_names[1]}={joy_female_pred[i][1]}")

Checking the intensity score average and bias

In [ ]:
diff_joy = []
diff_joy_count = 0  # number of pairs with differences
same_joy_count = 0  # number of pairs without differences
joy_male_prob_sum=0
joy_female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(joy_female_pred)):
          female_prob = joy_female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = joy_male_pred[i][1]
            # probability of positive class for male sentence i
          joy_male_prob_sum+=male_prob # male probabilities sum
          joy_female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diff_joy.append(diff)
          if diff > 0:
              diff_joy_count += 1
          else:
              same_joy_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_joy_count > 0:
    avg_diff = sum(diff_joy) / diff_joy_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_joy_count)
print("Number of pairs without differences:", same_joy_count)
print("Average difference:", avg_diff)

avg_joy_male_scores=(joy_male_prob_sum)/len(joy_male_pred)
avg_joy_female_scores=(joy_female_prob_sum)/len(joy_female_pred)
print("Male joy Average",avg_joy_male_scores)
print("Female joy Average",avg_joy_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_joy_female_scores > avg_joy_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# Fear Bias for male and female

fear male dataset

In [ ]:
new_fear_male=fear_male_data[['Sentence']]
new_fear_male.shape
fear_male_test = count_vectorizer.transform(new_fear_male['Sentence'])
fear_male_pred = pipeline_count.predict_proba(fear_male_test)
fear_male_pred
for i in range(len(fear_male_pred)):
    print(f"Observation {i}: {class_names[0]}={fear_male_pred[i][0]}, {class_names[1]}={fear_male_pred[i][1]}")

fear female dataset

In [ ]:
new_fear_female=fear_female_data[['Sentence']]
fear_female_test= count_vectorizer.transform(new_fear_female['Sentence'])
fear_female_pred= pipeline_count.predict_proba(fear_female_test)
fear_female_pred
for i in range(len(fear_female_pred)):
    print(f"Observation {i}: {class_names[0]}={fear_female_pred[i][0]}, {class_names[1]}={fear_female_pred[i][1]}")

Checking the intensity score average and bias

In [ ]:
diff_fear = []
diff_fear_count = 0  # number of pairs with differences
same_fear_count = 0  # number of pairs without differences
fear_male_prob_sum=0
fear_female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(fear_female_pred)):
          female_prob = fear_female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = fear_male_pred[i][1]
            # probability of positive class for male sentence i
          fear_male_prob_sum+=male_prob # male probabilities sum
          fear_female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diff_fear.append(diff)
          if diff > 0:
              diff_fear_count += 1
          else:
              same_fear_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_joy_count > 0:
    avg_diff = sum(diff_fear) / diff_fear_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_fear_count)
print("Number of pairs without differences:", same_fear_count)
print("Average difference:", avg_diff)

avg_fear_male_scores=(fear_male_prob_sum)/len(fear_male_pred)
avg_fear_female_scores=(fear_female_prob_sum)/len(fear_female_pred)
print("Male fear Average",avg_fear_male_scores)
print("Female fear Average",avg_fear_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_fear_female_scores > avg_fear_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# Non- Emotion Male and Female datasets

In [ ]:
# data_null_emotion = data[data['Emotion'].isnull()]

# data_null_emotion.shape

# # Group the data by gender
# No_emotion_gender_groups = data_null_emotion.groupby('Gender')

# # Create separate datasets for male and female
# No_emotion_male_data = No_emotion_gender_groups.get_group('male')
# No_emotion_female_data = No_emotion_gender_groups.get_group('female')
# print(No_emotion_female_data.shape)
# print(No_emotion_male_data.shape)

In [ ]:
male_test = count_vectorizer.transform(No_emotion_male_data['Sentence'])
male_pred =pipeline_count.predict_proba(male_test)
male_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(male_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

In [ ]:
female_test = count_vectorizer.transform(No_emotion_female_data['Sentence'])
female_pred = pipeline_count.predict_proba(female_test)
female_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

In [ ]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print("The Average of male",avg_male_scores)
print("The Average of female",avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# **Race Bias using CountVectorizer**

AfA Dataset

In [ ]:
Euro_data['Emotion'].isnull().sum()

In [ ]:
AfA_test = count_vectorizer.transform(AfA_data['Sentence'])
AfA_pred = pipeline_count.predict_proba(AfA_test)
AfA_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={AfA_pred[i][0]}, {class_names[1]}={AfA_pred[i][1]}")

Euro Dataset

In [ ]:
Euro_test=count_vectorizer.transform(Euro_data['Sentence'])
Euro_pred=pipeline_count.predict_proba(Euro_test)
Euro_pred
for i in range(len(Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={Euro_pred[i][0]}, {class_names[1]}={Euro_pred[i][1]}")

Checking the intensity score average and bias

In [ ]:
diff_list = []
diff_count_race = 0  # number of pairs with differences
same_count_race = 0  # number of pairs without differences
Euro_prob_sum=0
AfA_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(Euro_pred)):
          Euro_prob = Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = AfA_pred[i][1]  # probability of positive class for AfA sentence i
          Euro_prob_sum+=Euro_prob # Euro probabilities sum
          AfA_prob_sum+=AfA_prob # AfA probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_list.append(diff)
          if diff > 0:
              diff_count_race += 1
          else:
              same_count_race += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count_race > 0:
    avg_diff_race = sum(diff_list) / diff_count_race
else:
    avg_diff_race = 0

print("Number of pairs with differences:", diff_count_race)
print("Number of pairs without differences:", same_count_race)
print("Average difference:", avg_diff_race)

In [ ]:
avg_AfA_scores=(AfA_prob_sum)/len(AfA_pred)
avg_Euro_scores=(Euro_prob_sum)/len(Euro_pred)

print(avg_AfA_scores)
print(avg_Euro_scores)

# Check if E=A not significant
if abs(avg_diff_race) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_Euro_scores > avg_AfA_scores:
    print("E↑–A↓ significant")
else:
    print("E↓–A↑ significant")

**Race**

# Anger Bias for Race

Anger AfA dataset

In [ ]:
new_anger_AfA=anger_AfA_data[['Sentence']]
new_anger_AfA.shape
anger_AfA_test = count_vectorizer.transform(new_anger_AfA['Sentence'])
anger_AfA_pred = pipeline_count.predict_proba(anger_AfA_test)
anger_AfA_pred
for i in range(len(anger_AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={anger_AfA_pred[i][0]}, {class_names[1]}={anger_AfA_pred[i][1]}")

Anger Euro dataset

In [ ]:
new_anger_Euro=anger_Euro_data[['Sentence']]
anger_Euro_test= count_vectorizer.transform(new_anger_Euro['Sentence'])
anger_Euro_pred= pipeline_count.predict_proba(anger_Euro_test)
anger_Euro_pred
for i in range(len(anger_Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={anger_Euro_pred[i][0]}, {class_names[1]}={anger_Euro_pred[i][1]}")


Checking the intensity score average and bias

In [ ]:
diff_anger = []
diff_anger_count = 0  # number of pairs with differences
same_anger_count = 0  # number of pairs without differences
anger_AfA_prob_sum=0
anger_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(anger_Euro_pred)):
          Euro_prob = anger_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = anger_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          anger_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          anger_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_anger.append(diff)
          if diff > 0:
              diff_anger_count += 1
          else:
              same_anger_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_anger_count > 0:
    avg_diff = sum(diff_anger) / diff_anger_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_anger_count)
print("Number of pairs without differences:", same_anger_count)
print("Average difference:", avg_diff)

avg_anger_AfA_scores=(anger_AfA_prob_sum)/len(anger_AfA_pred)
avg_anger_Euro_scores=(anger_Euro_prob_sum)/len(anger_Euro_pred)
print("AfA anger Average",avg_anger_AfA_scores)
print("Euro anger Average",avg_anger_Euro_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_anger_Euro_scores > avg_anger_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

# Fear Bias for Race

fear AfA dataset

In [ ]:
new_fear_AfA=fear_AfA_data[['Sentence']]
new_fear_AfA.shape
fear_AfA_test = count_vectorizer.transform(new_fear_AfA['Sentence'])
fear_AfA_pred = pipeline_count.predict_proba(fear_AfA_test)
fear_AfA_pred
for i in range(len(fear_AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={fear_AfA_pred[i][0]}, {class_names[1]}={fear_AfA_pred[i][1]}")

Fear Euro dataset

In [ ]:
new_fear_Euro=fear_Euro_data[['Sentence']]
fear_Euro_test= count_vectorizer.transform(new_fear_Euro['Sentence'])
fear_Euro_pred= pipeline_count.predict_proba(fear_Euro_test)
fear_Euro_pred
for i in range(len(fear_Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={fear_Euro_pred[i][0]}, {class_names[1]}={fear_Euro_pred[i][1]}")



Checking the intensity score average and bias

In [ ]:
diff_fear = []
diff_fear_count = 0  # number of pairs with differences
same_fear_count = 0  # number of pairs without differences
fear_AfA_prob_sum=0
fear_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(fear_Euro_pred)):
          Euro_prob = fear_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = fear_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          fear_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          fear_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_fear.append(diff)
          if diff > 0:
              diff_fear_count += 1
          else:
              same_fear_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_fear_count > 0:
    avg_diff = sum(diff_fear) / diff_fear_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_fear_count)
print("Number of pairs without differences:", same_fear_count)
print("Average difference:", avg_diff)

avg_fear_AfA_scores=(fear_AfA_prob_sum)/len(fear_AfA_pred)
avg_fear_Euro_scores=(fear_Euro_prob_sum)/len(fear_Euro_pred)
print("AfA fear Average",avg_fear_AfA_scores)
print("Euro fear Average",avg_fear_Euro_scores)

# Check if E=A not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_fear_Euro_scores > avg_fear_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

# Sadness Bias for Race

sadness AfA dataset

In [ ]:
new_sadness_AfA=sadness_AfA_data[['Sentence']]
new_sadness_AfA.shape
sadness_AfA_test = count_vectorizer.transform(new_sadness_AfA['Sentence'])
sadness_AfA_pred = pipeline_count.predict_proba(sadness_AfA_test)
sadness_AfA_pred
for i in range(len(sadness_AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={sadness_AfA_pred[i][0]}, {class_names[1]}={sadness_AfA_pred[i][1]}")

sadness Euro dataset

In [ ]:
new_sadness_Euro=sadness_Euro_data[['Sentence']]
sadness_Euro_test= count_vectorizer.transform(new_sadness_Euro['Sentence'])
sadness_Euro_pred= pipeline_count.predict_proba(sadness_Euro_test)
sadness_Euro_pred
for i in range(len(sadness_Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={sadness_Euro_pred[i][0]}, {class_names[1]}={sadness_Euro_pred[i][1]}")


Checking the intensity score average and bias

In [ ]:
diff_sadness = []
diff_sadness_count = 0  # number of pairs with differences
same_sadness_count = 0  # number of pairs without differences
sadness_AfA_prob_sum=0
sadness_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(sadness_Euro_pred)):
          Euro_prob = sadness_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = sadness_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          sadness_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          sadness_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_sadness.append(diff)
          if diff > 0:
              diff_sadness_count += 1
          else:
              same_sadness_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_sadness_count > 0:
    avg_diff = sum(diff_sadness) / diff_sadness_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_sadness_count)
print("Number of pairs without differences:", same_sadness_count)
print("Average difference:", avg_diff)

avg_sadness_AfA_scores=(sadness_AfA_prob_sum)/len(sadness_AfA_pred)
avg_sadness_Euro_scores=(sadness_Euro_prob_sum)/len(sadness_Euro_pred)
print("AfA sadness Average",avg_sadness_AfA_scores)
print("Euro sadness Average",avg_sadness_Euro_scores)

# Check if E=A not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_sadness_Euro_scores > avg_sadness_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

# Joy Bias for Race

Joy AfA dataset

In [ ]:
new_joy_AfA=joy_AfA_data[['Sentence']]
new_joy_AfA.shape
joy_AfA_test = count_vectorizer.transform(new_joy_AfA['Sentence'])
joy_AfA_pred = pipeline_count.predict_proba(joy_AfA_test)
joy_AfA_pred
for i in range(len(joy_AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={joy_AfA_pred[i][0]}, {class_names[1]}={joy_AfA_pred[i][1]}")

Joy Euro dataset

In [ ]:
new_joy_Euro=joy_Euro_data[['Sentence']]
joy_Euro_test= count_vectorizer.transform(new_joy_Euro['Sentence'])
joy_Euro_pred= pipeline_count.predict_proba(joy_Euro_test)
joy_Euro_pred
for i in range(len(joy_Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={joy_Euro_pred[i][0]}, {class_names[1]}={joy_Euro_pred[i][1]}")


Checking the intensity score average and Bias

In [ ]:
diff_joy = []
diff_joy_count = 0  # number of pairs with differences
same_joy_count = 0  # number of pairs without differences
joy_AfA_prob_sum=0
joy_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(joy_Euro_pred)):
          Euro_prob = joy_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = joy_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          joy_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          joy_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_joy.append(diff)
          if diff > 0:
              diff_joy_count += 1
          else:
              same_joy_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_joy_count > 0:
    avg_diff = sum(diff_joy) / diff_joy_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_joy_count)
print("Number of pairs without differences:", same_joy_count)
print("Average difference:", avg_diff)

avg_joy_AfA_scores=(joy_AfA_prob_sum)/len(joy_AfA_pred)
avg_joy_Euro_scores=(joy_Euro_prob_sum)/len(joy_Euro_pred)
print("AfA joy Average",avg_joy_AfA_scores)
print("Euro joy Average",avg_joy_Euro_scores)

# Check if E=A not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_joy_Euro_scores > avg_joy_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

# No Race and Group by Gender Datasets

In [ ]:
male_test = count_vectorizer.transform(No_race_male_data['Sentence'])
male_pred = pipeline_count.predict_proba(male_test)
print(male_pred)
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(male_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

In [ ]:
female_test = count_vectorizer.transform(No_race_female_data['Sentence'])
female_pred = pipeline_count.predict_proba(female_test)
female_pr=pipeline_count.predict(female_test)
print(female_pr)
female_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

In [ ]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print("The Average of male",avg_male_scores)
print("The Average of female",avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

## **without Race (male and female datasets) with emotions**

In [ ]:
No_Race_with_emotion_male_group=No_race_male_data.groupby('Emotion')
No_Race_with_emotion_female_group=No_race_female_data.groupby('Emotion')


No_Race_with_anger_male_group=No_Race_with_emotion_male_group.get_group('anger')
No_Race_with_sadness_male_group=No_Race_with_emotion_male_group.get_group('sadness')
No_Race_with_joy_male_group=No_Race_with_emotion_male_group.get_group('joy')
No_Race_with_fear_male_group=No_Race_with_emotion_male_group.get_group('fear')

No_Race_with_anger_female_group=No_Race_with_emotion_female_group.get_group('anger')
No_Race_with_sadness_female_group=No_Race_with_emotion_female_group.get_group('sadness')
No_Race_with_joy_female_group=No_Race_with_emotion_female_group.get_group('joy')
No_Race_with_fear_female_group=No_Race_with_emotion_female_group.get_group('fear')

print(No_Race_with_anger_female_group.shape)
print(No_Race_with_anger_male_group.shape)

Anger male and female without race

In [ ]:
male_test = count_vectorizer.transform(No_Race_with_anger_male_group['Sentence'])
male_pred = pipeline_count.predict_proba(male_test)
male_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(male_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

In [ ]:
female_test = count_vectorizer.transform(No_Race_with_anger_female_group['Sentence'])
female_pred = pipeline_count.predict_proba(female_test)
female_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

In [ ]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print("The Average of male",avg_male_scores)
print("The Average of female",avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

Sadness male and female without Race

In [ ]:
male_test = count_vectorizer.transform(No_Race_with_sadness_male_group['Sentence'])
male_pred = pipeline_count.predict_proba(male_test)
male_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(male_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

In [ ]:
female_test = count_vectorizer.transform(No_Race_with_sadness_female_group['Sentence'])
female_pred = pipeline_count.predict_proba(female_test)
female_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

In [ ]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print("The Average of male",avg_male_scores)
print("The Average of female",avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

Joy male and female without Race

In [ ]:
male_test = count_vectorizer.transform(No_Race_with_joy_male_group['Sentence'])
male_pred = pipeline_count.predict_proba(male_test)
male_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(male_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

In [ ]:
female_test = count_vectorizer.transform(No_Race_with_joy_female_group['Sentence'])
female_pred = pipeline_count.predict_proba(female_test)
female_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

In [ ]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print("The Average of male",avg_male_scores)
print("The Average of female",avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

fear male and female without Race

In [ ]:
male_test = count_vectorizer.transform(No_Race_with_fear_male_group['Sentence'])
male_pred = pipeline_count.predict_proba(male_test)
male_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(lenmale_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

In [ ]:
female_test = count_vectorizer.transform(No_Race_with_fear_female_group['Sentence'])
female_pred = pipeline_count.predict_proba(female_test)
female_pred
class_names = pipeline_count.classes_
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

In [ ]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print("The Average of male",avg_male_scores)
print("The Average of female",avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

# Gender Bias and Race bias(Male-Male/Female-Female/AA-E)
Group by emotion and race and gender


In [ ]:
def gender_compare(set_name, male_set, female_set):
    male_set=count_vectorizer.transform(male_set['Sentence'])
    female_set=count_vectorizer.transform(female_set['Sentence'])   
    predictions_male = pipeline_count.predict_proba(male_set)
    male_pre=pipeline_count.predict(male_set)
    predictions_female = pipeline_count.predict_proba(female_set)
    female_pre=pipeline_count.predict(female_set)

    preds_male =male_pre
    # scores_male = np.argmax(predictions_male, axis=1)
    scores_male=np.maximum(predictions_male[:,0], predictions_male[:,1])

    preds_female = female_pre
    # scores_female = np.argmax(predictions_female, axis=1)
    scores_female=np.maximum(predictions_female[:,0], predictions_female[:,1])

    df = pd.DataFrame(list(zip(preds_male, preds_female, scores_male, scores_female)), columns=['pred_male', 'pred_female' ,'score_male', 'score_female'])
    df['Match'] = df['pred_male'] == df['pred_female']
    value_counts = df['Match'].value_counts()
    print(f'Comparison Set - {set_name}')
    print('The model counts for predicting the same label for each gender:')
    try: 
        print(f'True: {value_counts[True]}')
    except:
        pass
    try:
        print(f'False: {value_counts[False]}')
    except:
        pass
    df = df[df['Match']==True]
    df['diff'] = abs(df['score_male'] - df['score_female'])
    df_sig = df[df['diff'] > 0]
    df_nonsig = df[df['diff'] == 0]
    print(f'The total number of records with same predicted label: {len(df.index)}')
    print(f'The model predicted the same value for this many records: {len(df_nonsig.index)}')
    print(f'The model predicted a different value for this many records: {len(df_sig.index)}')
    print(f"Male average: {df_sig.describe().loc['mean', 'score_male']}")
    print(f"Female average: {df_sig.describe().loc['mean', 'score_female']}")
    print(f"Average difference: {df_sig.describe().loc['mean', 'diff']}")
    print('-'*25)
    print()



In [ ]:
def race_compare(set_name, AA_set, E_set):
    AA_set=count_vectorizer.transform(AA_set['Sentence'])
    E_set=count_vectorizer.transform(E_set['Sentence'])    
    predictions_AA = pipeline_count.predict_proba(AA_set)
    
    predictions_E = pipeline_count.predict_proba(E_set)

    preds_AA = pipeline_count.predict(AA_set)
    scores_AA = np.maximum(predictions_AA[:,0], predictions_AA[:,1])

    preds_E = pipeline_count.predict(E_set)
    scores_E = np.maximum(predictions_E[:,0], predictions_E[:,1])

    df = pd.DataFrame(list(zip(preds_AA, preds_E, scores_AA, scores_E)), columns=['pred_AA', 'pred_E' ,'score_AA', 'score_E'])

    df['Match'] = df['pred_AA'] == df['pred_E']
    value_counts = df['Match'].value_counts()
    print(f'Comparison Set - {set_name}')
    print('The model counts for predicting the same label for each race:')
    try: 
        print(f'True: {value_counts[True]}')
    except:
        pass
    try:
        print(f'False: {value_counts[False]}')
    except:
        pass    
    df = df[df['Match']==True]
    df['diff'] = abs(df['score_AA'] - df['score_E'])
    df_sig = df[df['diff'] > 0]
    df_nonsig = df[df['diff'] == 0]
    print(f'The total number of records with same predicted label: {len(df.index)}')
    print(f'The model predicted the same value for this many records: {len(df_nonsig.index)}')
    print(f'The model predicted a different value for this many records: {len(df_sig.index)}')
    print(f"African-American average: {df_sig.describe().loc['mean', 'score_AA']}")
    print(f"European average: {df_sig.describe().loc['mean', 'score_E']}")
    print(f"Average difference: {df_sig.describe().loc['mean', 'diff']}")
    print('-'*25)
    print()

In [ ]:
drop_columns = ['ID', 'Template', 'Emotion word', 'Person']
data.drop(columns=drop_columns, axis=1, inplace=True)


In [ ]:
non_race_df = data[data['Race'].isna()].dropna(subset=['Emotion'])
non_race_df

,Sentence,Gender,Race,Emotion
400,he feels angry.,male,NaN,anger
401,he feels furious.,male,NaN,anger
402,he feels irritated.,male,NaN,anger
403,he feels enraged.,male,NaN,anger
404,he feels annoyed.,male,NaN,anger
...,...,...,...,...
8635,the conversation with my mom was funny.,female,NaN,joy
8636,the conversation with my mom was hilarious.,female,NaN,joy
8637,the conversation with my mom was amazing.,female,NaN,joy
8638,the conversation with my mom was wonderful.,female,NaN,joy


In [ ]:
non_emotion_df = data[data['Emotion'].isna()].dropna(subset=['Race'])
non_emotion_df

,Sentence,Gender,Race,Emotion
4800,i saw alonzo in the market.,male,African-American,NaN
4801,i saw jamel in the market.,male,African-American,NaN
4802,i saw alphonse in the market.,male,African-American,NaN
4803,i saw jerome in the market.,male,African-American,NaN
4804,i saw leroy in the market.,male,African-American,NaN
...,...,...,...,...
5025,betsy has two children.,female,European,NaN
5026,kristin has two children.,female,European,NaN
5027,nancy has two children.,female,European,NaN
5028,stephanie has two children.,female,European,NaN


In [ ]:
non_emotion_non_race_df = data[data['Race'].isna() & data['Emotion'].isna()]
non_emotion_non_race_df

,Sentence,Gender,Race,Emotion
4820,i saw him in the market.,male,NaN,NaN
4821,i saw this man in the market.,male,NaN,NaN
4822,i saw this boy in the market.,male,NaN,NaN
4823,i saw my brother in the market.,male,NaN,NaN
4824,i saw my son in the market.,male,NaN,NaN
...,...,...,...,...
5035,my wife has two children.,female,NaN,NaN
5036,my girlfriend has two children.,female,NaN,NaN
5037,my mother has two children.,female,NaN,NaN
5038,my aunt has two children.,female,NaN,NaN


In [ ]:
data.dropna(inplace=True)


In [ ]:
grouped_emotion = data.groupby(['Emotion', 'Gender', 'Race'])
grouped_emotion_non_race = non_race_df.groupby(['Emotion', 'Gender'])

grouped_gender_non_emotion_non_race = non_emotion_non_race_df.groupby('Gender')
grouped_non_emotion = non_emotion_df.groupby(['Race', 'Gender'])

In [ ]:
df_female_AA_non_emotion = grouped_non_emotion.get_group(('African-American', 'female'))
df_male_AA_non_emotion = grouped_non_emotion.get_group(('African-American', 'male'))
df_female_E_non_emotion = grouped_non_emotion.get_group(('European', 'female'))
df_male_E_non_emotion = grouped_non_emotion.get_group(('European', 'male'))

df_female_non_emotion = grouped_gender_non_emotion_non_race.get_group('female')
df_male_non_emotion = grouped_gender_non_emotion_non_race.get_group('male')

print(f'AA female non-emotion size: {len(df_female_AA_non_emotion.index)}')
print(f'AA male non-emotion size: {len(df_male_AA_non_emotion.index)}')
print(f'E female non-emotion size: {len(df_female_E_non_emotion.index)}')
print(f'E male non-emotion size: {len(df_male_E_non_emotion.index)}')
print(f'Non-race female non-emotion size: {len(df_female_non_emotion.index)}')
print(f'Non-race male non-emotion size: {len(df_male_non_emotion.index)}')


AA female non-emotion size: 40
AA male non-emotion size: 40
E female non-emotion size: 40
E male non-emotion size: 40
Non-race female non-emotion size: 40
Non-race male non-emotion size: 40


In [ ]:
df_female_angry_non_race = grouped_emotion_non_race.get_group(('anger', 'female'))
df_female_fear_non_race = grouped_emotion_non_race.get_group(('fear', 'female'))
df_female_joy_non_race = grouped_emotion_non_race.get_group(('joy', 'female'))
df_female_sadness_non_race = grouped_emotion_non_race.get_group(('sadness', 'female'))

df_male_angry_non_race = grouped_emotion_non_race.get_group(('anger', 'male'))
df_male_fear_non_race = grouped_emotion_non_race.get_group(('fear', 'male'))
df_male_joy_non_race = grouped_emotion_non_race.get_group(('joy', 'male'))
df_male_sadness_non_race = grouped_emotion_non_race.get_group(('sadness', 'male'))

print(f'Non-race female angry size: {len(df_female_angry_non_race.index)}')
print(f'Non-race female fear size: {len(df_female_fear_non_race.index)}')
print(f'Non-race female joy size: {len(df_female_joy_non_race.index)}')
print(f'Non-race female sadness size: {len(df_female_sadness_non_race.index)}')
print(f'Non-race male angry size: {len(df_male_angry_non_race.index)}')
print(f'Non-race male fear size: {len(df_male_fear_non_race.index)}')
print(f'Non-race male joy size: {len(df_male_joy_non_race.index)}')
print(f'Non-race male sadness size: {len(df_male_sadness_non_race.index)}')

Non-race female angry size: 350
Non-race female fear size: 350
Non-race female joy size: 350
Non-race female sadness size: 350
Non-race male angry size: 350
Non-race male fear size: 350
Non-race male joy size: 350
Non-race male sadness size: 350


In [ ]:
df_female_angry_AA = grouped_emotion.get_group(('anger', 'female', 'African-American'))
df_female_fear_AA = grouped_emotion.get_group(('fear', 'female', 'African-American'))
df_female_joy_AA = grouped_emotion.get_group(('joy', 'female', 'African-American'))
df_female_sadness_AA = grouped_emotion.get_group(('sadness', 'female', 'African-American'))

df_male_angry_AA = grouped_emotion.get_group(('anger', 'male', 'African-American'))
df_male_fear_AA = grouped_emotion.get_group(('fear', 'male', 'African-American'))
df_male_joy_AA = grouped_emotion.get_group(('joy', 'male', 'African-American'))
df_male_sadness_AA = grouped_emotion.get_group(('sadness', 'male', 'African-American'))

df_female_angry_E = grouped_emotion.get_group(('anger', 'female', 'European'))
df_female_fear_E = grouped_emotion.get_group(('fear', 'female', 'European'))
df_female_joy_E = grouped_emotion.get_group(('joy', 'female', 'European'))
df_female_sadness_E = grouped_emotion.get_group(('sadness', 'female', 'European'))

df_male_angry_E = grouped_emotion.get_group(('anger', 'male', 'European'))
df_male_fear_E = grouped_emotion.get_group(('fear', 'male', 'European'))
df_male_joy_E = grouped_emotion.get_group(('joy', 'male', 'European'))
df_male_sadness_E = grouped_emotion.get_group(('sadness', 'male', 'European'))

print(f'AA female angry size: {len(df_female_angry_AA.index)}')
print(f'AA female fear size: {len(df_female_fear_AA.index)}')
print(f'AA female joy size: {len(df_female_joy_AA.index)}')
print(f'AA female sadness size: {len(df_female_sadness_AA.index)}')
print(f'AA male angry size: {len(df_male_angry_AA.index)}')
print(f'AA male fear size: {len(df_male_fear_AA.index)}')
print(f'AA male joy size: {len(df_male_joy_AA.index)}')
print(f'AA male sadness size: {len(df_male_sadness_AA.index)}')

print(f'E female angry size: {len(df_female_angry_E.index)}')
print(f'E female fear size: {len(df_female_fear_E.index)}')
print(f'E female joy size: {len(df_female_joy_E.index)}')
print(f'E female sadness size: {len(df_female_sadness_E.index)}')
print(f'E male angry size: {len(df_male_angry_E.index)}')
print(f'E male fear size: {len(df_male_fear_E.index)}')
print(f'E male joy size: {len(df_male_joy_E.index)}')
print(f'E male sadness size: {len(df_male_sadness_E.index)}')

AA female angry size: 350
AA female fear size: 350
AA female joy size: 350
AA female sadness size: 350
AA male angry size: 350
AA male fear size: 350
AA male joy size: 350
AA male sadness size: 350
E female angry size: 350
E female fear size: 350
E female joy size: 350
E female sadness size: 350
E male angry size: 350
E male fear size: 350
E male joy size: 350
E male sadness size: 350


In [ ]:
gender_compare('angry_AA', df_male_angry_AA, df_female_angry_AA)
gender_compare('fear_AA', df_male_fear_AA, df_female_fear_AA)
gender_compare('joy_AA', df_male_joy_AA, df_female_joy_AA)
gender_compare('sadness_AA', df_male_sadness_AA, df_female_sadness_AA)

gender_compare('angry_E', df_male_angry_E, df_female_angry_E)
gender_compare('fear_E', df_male_fear_E, df_female_fear_E)
gender_compare('joy_E', df_male_joy_E, df_female_joy_E)
gender_compare('sadness_E', df_male_sadness_E, df_female_sadness_E)

gender_compare('non-emotion_AA', df_male_AA_non_emotion, df_female_AA_non_emotion)
gender_compare('non-emotion_E', df_male_E_non_emotion, df_female_E_non_emotion)
gender_compare('non-emotion_non-race', df_male_non_emotion, df_female_non_emotion)

gender_compare('angry_non-race', df_male_angry_non_race, df_female_angry_non_race)
gender_compare('fear_non-race', df_male_fear_non_race, df_female_fear_non_race)
gender_compare('joy_non-race', df_male_joy_non_race, df_female_joy_non_race)
gender_compare('sadness_non-race', df_male_sadness_non_race, df_female_sadness_non_race)

Comparison Set - angry_AA
The model counts for predicting the same label for each gender:
True: 345
False: 5
The total number of records with same predicted label: 345
The model predicted the same value for this many records: 280
The model predicted a different value for this many records: 65
Male average: 0.60496330323714
Female average: 0.6081662628542301
Average difference: 0.01733863942105672
-------------------------

Comparison Set - fear_AA
The model counts for predicting the same label for each gender:
True: 345
False: 5
The total number of records with same predicted label: 345
The model predicted the same value for this many records: 280
The model predicted a different value for this many records: 65
Male average: 0.5895105936895925
Female average: 0.5921464483334775
Average difference: 0.017862142736015888
-------------------------

Comparison Set - joy_AA
The model counts for predicting the same label for each gender:
True: 349
False: 1
The total number of records with same

In [ ]:
race_compare('angry_male', df_male_angry_AA, df_male_angry_E)
race_compare('fear_male', df_male_fear_AA, df_male_fear_E)
race_compare('joy_male', df_male_joy_AA, df_male_joy_E)
race_compare('sadness_male', df_male_sadness_AA, df_male_sadness_E)

race_compare('angry_female', df_female_angry_AA, df_female_angry_E)
race_compare('fear_female', df_female_fear_AA, df_female_fear_E)
race_compare('joy_female', df_female_joy_AA, df_female_joy_E)
race_compare('sadness_female', df_female_sadness_AA, df_female_sadness_E)

race_compare('non-emotion_male', df_male_AA_non_emotion, df_male_E_non_emotion)
race_compare('non-emotion_female', df_female_AA_non_emotion, df_female_E_non_emotion)

Comparison Set - angry_male
The model counts for predicting the same label for each race:
True: 299
False: 51
The total number of records with same predicted label: 299
The model predicted the same value for this many records: 35
The model predicted a different value for this many records: 264
African-American average: 0.6112320678375913
European average: 0.6221137148914955
Average difference: 0.037618747524531884
-------------------------

Comparison Set - fear_male
The model counts for predicting the same label for each race:
True: 296
False: 54
The total number of records with same predicted label: 296
The model predicted the same value for this many records: 35
The model predicted a different value for this many records: 261
African-American average: 0.5921191279585503
European average: 0.6057881657970211
Average difference: 0.03726293963717146
-------------------------

Comparison Set - joy_male
The model counts for predicting the same label for each race:
True: 328
False: 22
The 